# Langchain crash course

In [1]:
import os
from pprint import pprint
import json

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## LLMs

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)


"Maharaja's Palace" or "Taj Mahal Bistro" 


In [4]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\n"Spice Palace"'

## Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))

{
    "cuisine": "Mexican",
    "text": "\n\n\"Cantina de Oro\" "
}


In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.
{
    "cuisine": "Mexican",
    "text": "\n\n\"La Casa Del Sabor\""
}


In [8]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [9]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

results = chain.invoke("Indian")
print(results["input"])
pprint(results["output"])

Indian
('\n'
 '\n'
 '1. Spicy Chicken Curry\n'
 '2. Vegetable Samosas\n'
 '3. Lamb Vindaloo\n'
 '4. Tandoori Chicken\n'
 '5. Aloo Gobi (Potato and Cauliflower Curry)\n'
 '6. Chana Masala (Chickpea Curry)\n'
 '7. Palak Paneer (Spinach and Cheese Curry)\n'
 '8. Naan Bread\n'
 '9. Mango Lassi (Mango Yogurt Drink)\n'
 '10. Vegetable Biryani\n'
 '11. Butter Chicken\n'
 '12. Onion Bhaji (Fried Onion Fritters)\n'
 '13. Rogan Josh (Lamb Stew)\n'
 '14. Dal Makhani (Creamy Lentil Curry)\n'
 '15. Gulab Jamun (Indian Sweet Dessert)')


#### Sequential Chain

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [11]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [12]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [13]:
results = chain.invoke({"cuisine": "Indian"})
pprint(results["cuisine"])
pprint(results["restaurant_name"])
pprint(results["menu_items"])

'Indian'
'\n"Spice Palace"'
('\n'
 '\n'
 '1. Butter Chicken: A classic North Indian dish made with tender chicken in a '
 'creamy and flavorful tomato-based sauce.\n'
 '\n'
 '2. Lamb Vindaloo: A spicy Goan curry made with chunks of tender lamb cooked '
 'in a tangy and spicy sauce.\n'
 '\n'
 '3. Vegetable Biryani: Fragrant basmati rice cooked with mixed vegetables, '
 'herbs, and spices.\n'
 '\n'
 '4. Tandoori Chicken: Marinated chicken cooked in a clay oven, served with a '
 'side of naan bread.\n'
 '\n'
 '5. Chana Masala: A popular vegetarian dish made with chickpeas cooked in a '
 'spicy and tangy tomato-based sauce.\n'
 '\n'
 '6. Palak Paneer: Creamy spinach and cottage cheese curry, a staple in North '
 'Indian cuisine.\n'
 '\n'
 '7. Dal Makhani: Slow-cooked black lentils in a rich and creamy tomato-based '
 'sauce.\n'
 '\n'
 '8. Chicken Tikka Masala: Grilled chicken pieces cooked in a creamy and '
 'aromatic tomato-based sauce.\n'
 '\n'
 '9. Aloo Gobi: A simple and flavorful dish

## Agents

In [14]:
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')

#### serpapi and llm-math tool

In [15]:
from langchain import hub
from langchain.agents import AgentType, create_react_agent, load_tools, AgentExecutor

In [16]:
prompt = hub.pull("hwchase17/react")

In [17]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What was the GDP of US in 2022?"

# Let's test it out!
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should search for the answer using a search engine.
Action: Search
Action Input: "GDP of US in 2022"$25.46 trillionI should use a calculator to convert the GDP from dollars to another currency if needed.
Action: Calculator
Action Input: $25.46 trillionAnswer: 25460000000000.0I now know the final answer.
Final Answer: The GDP of US in 2022 was $25.46 trillion.

> Finished chain.


{'input': 'What was the GDP of US in 2022?',
 'output': 'The GDP of US in 2022 was $25.46 trillion.'}

#### Wikipedia and llm-math tool

In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "When was Elon musk born? What is the square root of his age as of 2023?"

# Let's test it out!
agent_executor.invoke(input={"input": query})

In [ ]:
7.211102550927978 ** 2

## Memory

#### ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

In [ ]:
name = chain.invoke("Arabic")
print(name)

In [ ]:
print(chain.memory.buffer)

#### ConversationChain

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

In [ ]:
convo.invoke("Who won the first cricket world cup?")

In [ ]:
convo.invoke("How much is 5+5?")

In [ ]:
convo.invoke("Who was the captain ofthe winning team?")

In [ ]:
print(convo.memory.buffer)

#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

In [ ]:
convo.invoke("How much is 5+5?")

In [ ]:
convo.invoke("Who was the captain of the winning team?")

## Streaming

In [ ]:
for chunk in convo.stream("Write me a song about goldfish on the moon"):
    print(chunk["history"])
    # print(chunk.content, end="", flush=True)

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import GPT4All

# Set up the prompt template
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# Set up the local instace of the LLM
local_path = "/Users/brianroepke/Library/Application Support/nomic.ai/GPT4All/mistral-7b-openorca.Q4_0.gguf"  # noqa: E501
callbacks = [StreamingStdOutCallbackHandler()]
memory = ConversationBufferMemory()
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True, streaming=True)  # noqa: E501
# llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

llm_chain = ConversationChain(llm=llm)

In [ ]:
for chunk in llm_chain.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)